In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#오선 제거 부분 코드는 https://hackids.tistory.com/119 를 참조했습니다

In [10]:

# Music 불러오기
music = cv2.imread('816.png', cv2.IMREAD_GRAYSCALE)

# Music reszie

music = cv2.resize(music , [720,1280])


# 이진화
t,music = cv2.threshold(music,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)


# 음표 남기기(다른 요소 제거)
mask = np.zeros(music.shape, np.uint8)  
cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(music)  # 레이블링

for i in range(1, cnt):
    x, y, w, h, area = stats[i]
    if w > music.shape[1] * 0.8:  # 보표 영역에만
        cv2.rectangle(mask, (x, y, w, h), (255, 0, 0), -1)

music= cv2.bitwise_and(music, mask)  # 보표 영역 추출

cv2.imshow('music',music)
cv2.waitKey()
cv2.destroyAllWindows()


In [3]:
# music_copy로 박자 인식하기

music2 = cv2.imread('sample7.png',cv2.IMREAD_GRAYSCALE)



_,music2 = cv2.threshold(music2,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)





#옴표만 남기기
mask = np.zeros(music2.shape, np.uint8)  # 보표 영역만 추출하기 위해 마스크 생성
cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(music2)  # 레이블링

for i in range(1, cnt):
    x, y, w, h, area = stats[i]
    if w > music2.shape[1] * 0.5:  # 보표 영역에만
        cv2.rectangle(mask, (x, y, w, h), (255, 0, 0), -1)



music2 = cv2.bitwise_and( music2, mask)  # 보표 영역 추출




#문단 나누기

# 오선 위치 파악
y = []

h , w = music2.shape
for row in range(h):
    pix = 0
    for col in range(w):
        if music2[row,col] == 255:
                pix +=1
    if pix > w * 0.5:
         y.append(row)

# 소절 분리
num_line = int(len(y) / 5)

num_collection =[]   # -> 각 문단의 y좌표를 저장하는 2차원 리스트

# num_line만큼 오선을 저장할 리스트 생성
for num in range(num_line):
     num_line_list = []
     num_collection.append(num_line_list)


#2차원 리스트에 문단정보 저장

gap_i =  0
paragraph_i = 0
while gap_i < len(y) -1:



    gap = abs(y[gap_i] - y[gap_i + 1])
    gap_i +=1

    if gap < 30:
        num_collection[paragraph_i].append(y[gap_i])
        
        
        
    elif gap >= 60:
        paragraph_i += 1

print(num_collection)
            
# 문단 저장 확인
for i in range(num_line):
     print("문단 {}".format(i + 1))
     print(num_collection[i])
     print("--------------------")

print("오선의 y좌표 ",y)



h , w = music2.shape
staves = []  

for row in range(h):
    pixels = 0
    for col in range(w):
        pixels += (music2[row][col] == 255)  # 한 행에 존재하는 흰색 픽셀의 개수를 셈
    if pixels >= w * 0.5:  # 이미지 넓이의 50% 이상이라면
        if len(staves) == 0 or abs(staves[-1][0] + staves[-1][1] - row) >= 1:  # 첫 오선이거나 이전에 검출된 오선과 다른 오선
            staves.append([row, 1])  # 오선 추가 [오선의 y 좌표][오선 높이]
        else:  
            staves[-1][1] += 1  
for staff in range(len(staves)):
    top_pixel = staves[staff][0]  
    bot_pixel = staves[staff][0] + staves[staff][1]  # 오선의 최하단 y 좌표 (오선의 최상단 y 좌표 + 오선 높이)
    for col in range(w):
        if music2[top_pixel - 1][col] == 0 and music2[bot_pixel + 1][col] == 0:  # 오선 위, 아래로 픽셀이 있는지 탐색
            for row in range(top_pixel, bot_pixel + 1):
                music2[row][col] = 0  # 오선을 지움

music2 = cv2.bitwise_not(music2)


for i , music_sheet in enumerate(num_collection):
    if i == 0:
        start_0 = music_sheet[0] -15
        end_0 = music_sheet[-1] +18
        M1 = music2[start_0 : end_0]
    elif i ==1:
        start_1 = music_sheet[0] -15
        end_1 = music_sheet[-1] +18
        M2 = music2[start_1 : end_1]
    elif i == 2:
        start_2 = music_sheet[0] -15
        end_2 = music_sheet[-1] +18
        M3 = music2[start_2 : end_2]
        
    elif i ==3:
        start_3 = music_sheet[0] -15
        end_3 = music_sheet[-1] +18
        M4 = music2[start_3 : end_3]




[[130, 138, 145, 146, 153], [252, 259, 260, 267, 275], [373, 374, 381, 389, 396], [488, 495, 503, 510, 518]]
문단 1
[130, 138, 145, 146, 153]
--------------------
문단 2
[252, 259, 260, 267, 275]
--------------------
문단 3
[373, 374, 381, 389, 396]
--------------------
문단 4
[488, 495, 503, 510, 518]
--------------------
오선의 y좌표  [123, 130, 138, 145, 146, 153, 244, 252, 259, 260, 267, 275, 366, 373, 374, 381, 389, 396, 487, 488, 495, 503, 510, 518]


In [4]:
# # 모폴로지 연산으로 지울거 지우고 남길거 남기기

def F_mor(music):
    
    k1 = cv2.getStructuringElement(cv2.MORPH_RECT, (1,1))
    mor1 = cv2.morphologyEx(music , cv2.MORPH_DILATE,kernel=k1, iterations=10)

    # k2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
    # mor2 = cv2.morphologyEx(mor1,cv2.MORPH_ERODE,kernel=k2 , iterations=2)

    # k3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE , (1,1))
    # mor3 = cv2.morphologyEx(mor2 , cv2.MORPH_ERODE , kernel= k3 ,iterations=1)

    return mor1

M1 = F_mor(M1)
M2 = F_mor(M2)
M3 = F_mor(M3)
M4 = F_mor(M4)


M1_copy = M1.copy()
M2_copy = M2.copy()
M3_copy = M3.copy()
M4_copy = M4.copy()



In [5]:
# 템플릿 매칭 하기


def temp(music,sorce):
    src = sorce
    temp = cv2.imread(src,cv2.IMREAD_GRAYSCALE)
    method = cv2.TM_CCOEFF_NORMED
    result = cv2.matchTemplate(music, temp, method)
    return result ,temp

# 2분음표

# 1문단
M1_R , temp_1= temp(M1,'M1_temp.png')


# 2문단
M2_R_1 , temp_2_1 = temp(M2,'M2_temp_1.png')
M2_R_2 , temp_2_2 = temp(M2,'M2_temp_2.png')


# 3문단
M3_R , temp_3 = temp(M3,'M3_temp.png')


# 4문단
M4_R_1 , temp_4_1 = temp(M4,'M4_temp_1.png')
M4_R_2 , temp_4_2 = temp(M4,'M4_temp_2.png')


# 4분음표




# 음표 찾기 
def Find_beat(music,reslut,temp,threshold):
    thr= threshold
    locations = np.where(reslut >= thr)
    for loc in zip(*locations[::-1]):
        cv2.rectangle(music, loc, (loc[0] + temp.shape[1], loc[1] + temp.shape[0]), (0, 255  , 0), 1)
    return locations

# 2분음표 
#1문단
half_M1_location = Find_beat(M1 , M1_R,temp_1,0.95)
half_x1 = [loc[0] for loc in half_M1_location]


#2문단
half_M2_location_1= Find_beat(M2 , M2_R_1,temp_2_1,0.9)
half_x2_1 = [loc[0] for loc in half_M2_location_1]

half_M2_location_2= Find_beat(M2 , M2_R_2,temp_2_2,0.9)
half_x2_2 = [loc[0] for loc in half_M2_location_2]


#3문단
half_M3_location= Find_beat(M3 , M3_R,temp_3,0.8)
half_x3 = [loc[0] for loc in half_M3_location]


#4문단

half_M4_location_1 =Find_beat(M4 , M4_R_1,temp_4_1,0.9)
half_x4_1 = [loc[0] for loc in half_M4_location_1]
half_M4_location_2 =Find_beat(M4 , M4_R_2,temp_4_2,0.85)
half_x4_2 = [loc[0] for loc in half_M4_location_2]


In [6]:
# 4분음표 인식을 위한 객체탐지
# M1_copy / M2_copy / M3_copy/ M4_copy/

def bit_not(Music):
    M = cv2.bitwise_not(Music)
    return M

M1_copy = bit_not(M1_copy)
M2_copy = bit_not(M2_copy)
M3_copy = bit_not(M3_copy)
M4_copy = bit_not(M4_copy)





# 모폴로지 연산
def F_mor2(music):


    k1 = cv2.getStructuringElement(cv2.MORPH_RECT , (3,3))
    mor1 = cv2.morphologyEx(music , cv2.MORPH_DILATE , k1 , iterations=2)
    k2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE , (3,3))
    mor2 = cv2.morphologyEx(mor1 , cv2.MORPH_ERODE , k2 , iterations=4)
    k3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,1))
    mor3 = cv2.morphologyEx(mor2 , cv2.MORPH_DILATE , k3 ,iterations= 2)
    k4 = cv2.getStructuringElement(cv2.MORPH_RECT , (2,2))
    mor4 = cv2.morphologyEx(mor3 , cv2.MORPH_ERODE , k4 , iterations=1)
    
    return mor4

M1_copy = F_mor2(M1_copy)
M2_copy = F_mor2(M2_copy)
M3_copy = F_mor2(M3_copy)
M4_copy = F_mor2(M4_copy)



# connectcompoent
def Find_compoent(Music):

    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(Music)  # 모든 객체 검출하기
    sorted_x = np.argsort(stats[2: , cv2.CC_STAT_LEFT] )  + 1
    for i in range(1, cnt):
        (x, y, w, h, area) = stats[i]
        

        cv2.rectangle(Music, (x, y, w, h), (255, 0, 0), 1)

    return cnt,stats

cnt_1,stats_1 = Find_compoent(M1_copy)
cnt_2,stats_2 = Find_compoent(M2_copy)
cnt_3,stats_3 = Find_compoent(M3_copy)
cnt_4,stats_4 = Find_compoent(M4_copy)

#왼쪽 위 모서리의 x 좌표 (x-coordinate of the top-left corner)
# 왼쪽 위 모서리의 y 좌표 (y-coordinate of the top-left corner)
# 구성 요소의 너비 (width of the component)
# 구성 요소의 높이 (height of the component)
# 구성 요소의 면적 (area of the component)

print("cnt = {}     /    stats ={} ".format(cnt_1, stats_1[2:,0]  ))
print("-------------------------------------------------------")
print("cnt = {}     /    stats = {} ".format(cnt_2  , stats_2[2:,0]))
print("-------------------------------------------------------")
print("cnt = {}     /    stats = {} ".format(cnt_3 , stats_3[2:,0]))
print("-------------------------------------------------------")
print("cnt = {}     /    stats = {}".format(cnt_4 , stats_4[2:,-0] ))

sort_x1 = sorted(stats_1[5:,0])
sort_x2 = sorted(stats_2[2:,0])
sort_x3 = sorted(stats_3[2:,0])
sort_x4 = sorted(stats_4[2:,0])

print("-------------------------------------------------")
print("sort_x1 = {} , 개수 = {}".format(sort_x1 , len(sort_x1)))
print(half_x1) 

print("-------------------------------------------------")
print("sort_x2 = {}".format(sort_x2))
print(half_x2_1)
print(half_x2_2)
print("-------------------------------------------------")
print("sort_x3 = {}".format(sort_x3))
print(half_x3)
print("-------------------------------------------------")
print("sort_x4 = {}".format(sort_x4))
print(half_x4_1)
print(half_x4_2)



print("-------------------------------------------------")




cnt = 16     /    stats =[ 53  53 243 277  52 157 190 335 415 449 502 536  72 104] 
-------------------------------------------------------
cnt = 12     /    stats = [226 264 320 357 411 448  57  91 506 145] 
-------------------------------------------------------
cnt = 13     /    stats = [536  56  89 503 175 142 227 265 322 405 443] 
-------------------------------------------------------
cnt = 13     /    stats = [567  57  91 143 227 261 315 350 407 442 496]
-------------------------------------------------
sort_x1 = [52, 72, 104, 157, 190, 277, 335, 415, 449, 502, 536] , 개수 = 11
[1, 320]
-------------------------------------------------
sort_x2 = [57, 91, 145, 226, 264, 320, 357, 411, 448, 506]
[15, 135]
[11, 498]
-------------------------------------------------
sort_x3 = [56, 89, 142, 175, 227, 265, 322, 405, 443, 503, 536]
[9, 311]
-------------------------------------------------
sort_x4 = [57, 91, 143, 227, 261, 315, 350, 407, 442, 496, 567]
[11, 132]
[23, 486]
---------------

In [7]:
# 리스트

bit_1  = []
bit_2 = []
bit_3 = []
bit_4 = []

def append_bit(cord ,bit , coordniate):
    if cord == 1:
        for i in coordniate:
            
            if i >=310 and i<=340:
                bit.append('0')
            else:
            
                bit.append('1')
    
    elif cord == 2:
        for i in coordniate:
            
            if i >=135 and i<=145:
                bit.append('0')
            elif i>=498 and i<=506:
                bit.append('0')

            else:
                bit.append('1')
            
    
    elif cord == 3:
        for i in coordniate:
            
            if i>=311 and i<=322:
                bit.append('0')
            else:
            
                bit.append('1')
    elif cord ==4:
        for i in coordniate:
            
            if i>=132 and i <=143:
                bit.append('0')
            elif i>=486 and i<=496:
                bit.append('0')
            else:
                bit.append('1')
        
        bit.pop()
    
    return bit
bit1 = append_bit(1,bit_1,sort_x1)
bit2 = append_bit(2,bit_2,sort_x2)
bit3 = append_bit(3,bit_3,sort_x3)
bit4 = append_bit(4,bit_4,sort_x4)







sum_bit = bit1 + bit2 +bit3 + bit4
print(bit1)
print(bit2) # ->마지막 제거
print(bit3) # -> 마지막 제거
print(bit4) # ->마지막 3개 이상

# print('------------------------------------------------------------------------')
print(sum_bit)





['1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1']
['1', '1', '0', '1', '1', '1', '1', '1', '1', '0']
['1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1']
['1', '1', '0', '1', '1', '1', '1', '1', '1', '0']
['1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0']


In [8]:
# 오선 제거
hist = np.zeros(music.shape, np.uint8)
hist_y = np.zeros((music.shape[0], music.shape[1], 3), np.uint8)  # 컬러 이미지로 초기화

h , w = music.shape

start_y = []
end_y = []

# 모폴로지
morhp1 = cv2.morphologyEx(music,cv2.MORPH_DILATE,kernel=None)
morhp2 = cv2.morphologyEx(morhp1,cv2.MORPH_ERODE,kernel=None,iterations=1)

music = morhp2
music_with_lines = morhp2.copy()



# 오선 위치 저장
for row in range(h):
    pixels = 0
    for col in range(w):
        pixels += (music[row][col] == 255)
    for pixel in range(pixels):
        hist[row][pixel] = 255

    if pixels > w*0.5:
        start_y.append([row,1])
        end_y.append(row)


# y오선 위치 파악
for y in end_y:
    cv2.line(hist_y, (0, y), (w - 1, y), (0,255,0), 1)





print('start_y',start_y)
print('end_y',end_y)
print('개수',len(end_y))

cv2.imshow('music',music)
cv2.waitKey()
cv2.destroyAllWindows()


start_y [[151, 1], [158, 1], [159, 1], [166, 1], [173, 1], [174, 1], [181, 1], [270, 1], [271, 1], [277, 1], [278, 1], [285, 1], [286, 1], [292, 1], [293, 1], [300, 1], [301, 1], [390, 1], [397, 1], [398, 1], [405, 1], [412, 1], [413, 1], [419, 1], [420, 1], [509, 1], [516, 1], [517, 1], [524, 1], [531, 1], [532, 1], [539, 1]]
end_y [151, 158, 159, 166, 173, 174, 181, 270, 271, 277, 278, 285, 286, 292, 293, 300, 301, 390, 397, 398, 405, 412, 413, 419, 420, 509, 516, 517, 524, 531, 532, 539]
개수 32


In [9]:
# 오선 개수 
num_y = len(end_y)

# 문단 개수
num_paragraph = int((num_y) / 5)




# 오선 개수 파악
line_num = len(end_y)

# Music_seperate
Code_1 = []
Code_2 = []
Code_3 = []
Code_4 = []
Music_sum = []



# 오선 묶어주기
# 오선 정보를 보면 오차가 90~100이상 나는 부분을 한 문단이 끝나는 부분으로 보고 그 부분을 기준선으로 오선을 묶어주기

print('----------------------')
line_1 = 0

while line_1 < line_num - 1:

    
    gap_1 = abs(end_y[line_1] - end_y[line_1+1])

    Code_1.append(end_y[line_1])
    
    if gap_1 >= 60:
        line_2 = line_1+1
        break
    line_1 +=1
    
while line_2 < line_num -1:

    
    gap_2 = abs(end_y[line_2]-end_y[line_2+1])
    Code_2.append(end_y[line_2])
        
    if gap_2 >=60:
        line_3 = line_2+1
        break
    line_2 +=1

while line_3 < line_num -1 :


    gap_3 = abs(end_y[line_3]-end_y[line_3+1])
    Code_3.append(end_y[line_3])
        
    if gap_3 >=60:
        line_4 = line_3 +1
        break
    line_3 +=1



while line_4 < line_num -1 :


    gap_4 = abs(end_y[line_4]-end_y[line_4+1])
    Code_4.append(end_y[line_4])
        
    if gap_4 >=90:
        
        break
    line_4 +=1

# 마지막 원소 처리
Code_4.append(end_y[-1])




print('music_1',Code_1)
print('music_2',Code_2)
print('music_3',Code_3)
print('music_4',Code_4)

l1 = len(Code_1)
l2 = len(Code_2)
l3 = len(Code_3)
l4 = len(Code_4)

l5 = len(end_y)
print('sum : , end_y = ',l1+l2+l3+l4 , l5)
print('end',end_y)



----------------------
music_1 [151, 158, 159, 166, 173, 174, 181]
music_2 [270, 271, 277, 278, 285, 286, 292, 293, 300, 301]
music_3 [390, 397, 398, 405, 412, 413, 419, 420]
music_4 [509, 516, 517, 524, 531, 532, 539]
sum : , end_y =  32 32
end [151, 158, 159, 166, 173, 174, 181, 270, 271, 277, 278, 285, 286, 292, 293, 300, 301, 390, 397, 398, 405, 412, 413, 419, 420, 509, 516, 517, 524, 531, 532, 539]


In [13]:
# 소절 정보로 악보 분리하기
# 연주할 문단 선택하기
paragraph = int(input('연주할 문단 선택(차례대로)'))
if paragraph > num_paragraph:
    print('다시 선택하세요')
    while(True):
        paragraph = input('연주할 문단 선택')
        if int(paragraph) < int(num_paragraph):
            break
else:
    pass

    
if paragraph == 1:
        
    Music_1 = music[Code_1[0] -5 : Code_1[-1] +25 ,:]
    cv2.imshow('Music_1', Music_1)
    cv2.imshow('Music',music)

    cv2.waitKey()
    cv2.destroyAllWindows()

if paragraph == 2:
    Music_2 = music[Code_2[0] -5 : Code_2[-1] + 25 , :]
    cv2.imshow('Music_2', Music_2)
    cv2.imshow('Music',music)

    cv2.waitKey()
    cv2.destroyAllWindows()

if paragraph == 3:
    Music_3 = music[Code_3[0] -5 : Code_3[-2] + 25 , :]
    cv2.imshow('Music_3', Music_3)
    cv2.imshow('Music',music)

    cv2.waitKey()
    cv2.destroyAllWindows()

if paragraph == 4:
    Music_4 = music[Code_4[0] -5  : Code_4[-1] +25 , :]
    cv2.imshow('Music_4', Music_4)
    cv2.imshow('Music',music)

    cv2.waitKey()
    cv2.destroyAllWindows()
 


In [14]:

# 악보 오선 제거
All_music = [Music_1 , Music_2 ,Music_3 , Music_4]


"""
def remove_staff_line(music):
    h , w = music.shape
    for row in range(h):
        pixels = 0
        for col in range(w):
            pixels += (music[row][col] == 255)  # 한 행에 존재하는 흰색 픽셀의 개수를 셈
        if pixels >= w * 0.5:  # 이미지 넓이의 50% 이상이라면
            for col in range(w):
                music[row][col] = 0  # 제거
"""
def remove_staff_line(music):
    h , w = music.shape
    staves = []  

    for row in range(h):
        pixels = 0
        for col in range(w):
            pixels += (music[row][col] == 255)  # 한 행에 존재하는 흰색 픽셀의 개수를 셈
        if pixels >= w * 0.5:  # 이미지 넓이의 50% 이상이라면
            if len(staves) == 0 or abs(staves[-1][0] + staves[-1][1] - row) >= 1:  # 첫 오선이거나 이전에 검출된 오선과 다른 오선
                staves.append([row, 1])  # 오선 추가 [오선의 y 좌표][오선 높이]
            else:  
                staves[-1][1] += 1  
    for staff in range(len(staves)):
        top_pixel = staves[staff][0]  
        bot_pixel = staves[staff][0] + staves[staff][1]  # 오선의 최하단 y 좌표 (오선의 최상단 y 좌표 + 오선 높이)
        for col in range(w):
            if music[top_pixel - 1][col] == 0 and music[bot_pixel + 1][col] == 0:  # 오선 위, 아래로 픽셀이 있는지 탐색
                for row in range(top_pixel, bot_pixel + 1):
                    music[row][col] = 0  # 오선을 지움


    return music

Music_1 = remove_staff_line(Music_1)
Music_2 = remove_staff_line(Music_2)
Music_3 = remove_staff_line(Music_3)
Music_4 = remove_staff_line(Music_4)


cv2.imshow('music1',Music_1)
cv2.imshow('music2',Music_2)
cv2.imshow('music3',Music_3)
cv2.imshow('music4',Music_4)
cv2.waitKey()
cv2.destroyAllWindows()

In [15]:
# resize Func

def resize_Music(music):
    resize_M = cv2.resize(music , [614,61])
    return resize_M

Re_Music_1 = resize_Music(Music_1)
Re_Music_2 = resize_Music(Music_2)
Re_Music_3 = resize_Music(Music_3)
Re_Music_4 = resize_Music(Music_4)


cv2.imshow('Re_Music_1',Re_Music_1)
cv2.imshow('Re_Music_2',Re_Music_2)
cv2.imshow('Re_Music_3',Re_Music_3)
cv2.imshow('Re_Music_4',Re_Music_4)
cv2.waitKey()
cv2.destroyAllWindows()

In [16]:
#1. 꼬리와 머리의 템플릿을 분리하기.
#2. 머리부터 템플릿 매칭
#3. 매칭된 모든 템플릿의 y 좌표 중심을 구하기
#4. 구한 y좌표의 중심을 비교해서 음계 매칭

def Morph(music):
    k = cv2.getStructuringElement(cv2.MORPH_RECT , (2,2))
    music = cv2.morphologyEx(music ,cv2.MORPH_CLOSE , kernel= k ,iterations= 3)
    return music

mor_Music_1 = Morph(Re_Music_1)
mor_Music_2 = Morph(Re_Music_2)
mor_Music_3 = Morph(Re_Music_3)
mor_Music_4 = Morph(Re_Music_4)


cv2.imshow('music_1',mor_Music_1)
cv2.imshow('music_2',mor_Music_2)
cv2.imshow('music_3',mor_Music_3)
cv2.imshow('music_4',mor_Music_4)

cv2.waitKey()
cv2.destroyAllWindows()


def Find_Music_head(music):
    Music_temp = cv2.imread('music_temp1.png',cv2.IMREAD_GRAYSCALE)
    res = cv2.matchTemplate(music ,Music_temp , cv2.TM_CCOEFF_NORMED)
    threshold = 0.75
    
    # locations -> [0] : 행(y) / [1]: 열(x) 
    locations = np.where(res >= threshold)
    music = cv2.cvtColor(music , cv2.COLOR_GRAY2BGR)
    

    #*은 unpacking 연산자로서, 시퀀스의 각 원소를 개별 인자로 전달하는 역할
    # -1을 넣은이유 : (y,x)를 (x,y)로 보려고
    for pt in zip(*locations[::-1]):
        bottom_right = (pt[0] + Music_temp.shape[1], pt[1] + Music_temp.shape[0])
        cv2.rectangle(music, pt, bottom_right, (0, 0, 255), 1)  # 사각형 그리기

    # X좌표를 기준으로 오름차순 정리하기 (x,y순)

    detected_objects = list(zip(*locations[::-1]))
    detected_objects.sort(key = lambda x : x[0])
    
    # 중복된 좌표 제거
    # 리스트 컴프레션 
    # new_list = [expression for element in iterable if condition]
    # expression : 표현 / for -> if 사용 문법은 그대로


    unique_x_coords = set()
    U_detected_objects = []
    U_detected_objects_Y = []

    for x, y in detected_objects:
        if all(abs(x - prev_x) > 10 for prev_x, _ in U_detected_objects):
            unique_x_coords.add(x)
            U_detected_objects.append((x, y))
            U_detected_objects_Y.append(y)

    
    
    print(U_detected_objects_Y)
    print("개수 = ",len(U_detected_objects))
    return music , U_detected_objects_Y ,U_detected_objects

F_Music_1 ,coordinate_1 ,Full_coordinate_1 =Find_Music_head(mor_Music_1)
F_Music_2 ,coordinate_2 , Full_coordinate_2=Find_Music_head(mor_Music_2)
F_Music_3 ,coordinate_3 ,Full_coordinate_3=Find_Music_head(mor_Music_3)
F_Music_4 ,coordinate_4 ,Full_coordinate_4=Find_Music_head(mor_Music_4)


cv2.imshow('Music_1',F_Music_1)

cv2.imshow('Music_2',F_Music_2)

cv2.imshow('Music_3',F_Music_3)

cv2.imshow('Music_4',F_Music_4)

cv2.waitKey()
cv2.destroyAllWindows()



[41, 40, 26, 25, 22, 22, 26, 29, 30, 33, 33]
개수 =  11
[36, 36, 41, 26, 26, 31, 31, 33, 34, 36]
개수 =  10
[25, 26, 30, 29, 34, 34, 36, 42, 42, 26, 25]
개수 =  11
[23, 22, 25, 29, 29, 33, 33, 36, 36, 41]
개수 =  10


In [18]:
# detected object

# F_Music_1(1문단) / F_Music_2(2문단) / F_Music_3(3문단) /F_Music_4(4문단)
# coordinate_1    / coordinate_2     / coordinate_3    / coordinate_4
def recognize_head(music , coordinate):
    Final_Music_code = []
    
    for i in coordinate:
        
        if i >=40 and i <=42:
            Final_Music_code.append("f")
            
        elif i >=35 and i<=37:
            Final_Music_code.append("g")
            
        elif i>=32 and i<=34:
            Final_Music_code.append('h')
            
        elif i>=28 and i<=31:
            Final_Music_code.append("i")
            
        elif i>=24 and i<=26:
            Final_Music_code.append("j")
            
        elif i>=21 and i<=23:
            Final_Music_code.append("k")
    
    print(Final_Music_code)
    return Final_Music_code

Final_Music_code_1 = recognize_head(F_Music_1 , coordinate_1)
Final_Music_code_2 = recognize_head(F_Music_2 , coordinate_2)
Final_Music_code_3 = recognize_head(F_Music_3 , coordinate_3)
Final_Music_code_4 = recognize_head(F_Music_4 , coordinate_4)


def Write_head(music , coordinate):
    Full_coordinate = coordinate
    
    for x,y in Full_coordinate:
        text = ""  # 텍스트 변수 초기화

        if y >= 40 and y <= 42:
            text = "Do"
            
        elif y >= 35 and y <= 37:
            text = "Re"
            

        elif y >= 32 and y <= 34:
            text = "Mi"
            
        elif y>=28 and y<=31:
            text = "Fa"
            
        elif y >= 24 and y <= 26:
            text = "So"
            
        elif y >= 21 and y <= 23:
            text = "La"
            
        
        if text:  # 텍스트가 할당되었을 때만 추가
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            font_color = (0, 255,0)
            cv2.putText(music, text, (x, y-15), font, font_scale, font_color , 2)
    
    cv2.imshow('music',music)
    cv2.waitKey()
    cv2.destroyAllWindows()
    return music

Final_Music_1 = Write_head(F_Music_1,Full_coordinate_1)
Final_Music_2 = Write_head(F_Music_2,Full_coordinate_2)
Final_Music_3 = Write_head(F_Music_3,Full_coordinate_3)
Final_Music_4 = Write_head(F_Music_4,Full_coordinate_4)

sum_fianl_coordinate = Final_Music_code_1 +Final_Music_code_2 +Final_Music_code_3 +Final_Music_code_4

print('----------------------------------------------------------')
print("Final _ Music _ Coordinate ")
print(sum_fianl_coordinate)

print('-----------------------------------------------------------')
print("Final _sum _bit")
print(sum_bit)


print('-------------------------------------------------------------')
print("bit의 개수 = ",len(sum_bit))
print("음계의 개수" , len(sum_fianl_coordinate))

['f', 'f', 'j', 'j', 'k', 'k', 'j', 'i', 'i', 'h', 'h']
['g', 'g', 'f', 'j', 'j', 'i', 'i', 'h', 'h', 'g']
['j', 'j', 'i', 'i', 'h', 'h', 'g', 'f', 'f', 'j', 'j']
['k', 'k', 'j', 'i', 'i', 'h', 'h', 'g', 'g', 'f']
----------------------------------------------------------
Final _ Music _ Coordinate 
['f', 'f', 'j', 'j', 'k', 'k', 'j', 'i', 'i', 'h', 'h', 'g', 'g', 'f', 'j', 'j', 'i', 'i', 'h', 'h', 'g', 'j', 'j', 'i', 'i', 'h', 'h', 'g', 'f', 'f', 'j', 'j', 'k', 'k', 'j', 'i', 'i', 'h', 'h', 'g', 'g', 'f']
-----------------------------------------------------------
Final _sum _bit
['1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '0']
-------------------------------------------------------------
bit의 개수 =  42
음계의 개수 42
